<a href="https://colab.research.google.com/github/mertcan-basut/nlp/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai

!pip install -q langchain langchain-openai

!pip install -q python-dotenv

In [2]:
!echo "OPENAI_API_KEY=editthis" > .env

In [57]:
# a framework for developing LM powered applications
from langchain_openai import ChatOpenAI # models
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, PromptTemplate # input prompts
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.output_parsers import StructuredOutputParser # output parsers
from langchain.output_parsers import ResponseSchema
from langchain.chains import ConversationChain # memory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationTokenBufferMemory, ConversationSummaryBufferMemory
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain # chains
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

import openai # direct API calls to OpenAI

import json
import os
# https://platform.openai.com/api-keys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')
# /usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.

## Models, Prompts, Parsers

In [ ]:
text = """\
This leaf blower is pretty amazing.  It has four settings: \
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.\
"""

system_message_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.
"""

human_message_template = """\
text: {text}
"""

format_instructions_template = """\
Format the output as JSON with the following keys:
gift
delivery_days
price_value
"""

In [ ]:
client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def get_completion(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.choices[0].message.content

get_completion("Hi!")

'Hello! How can I assist you today?'

In [ ]:
def format_prompt(system_message, human_message, format_instructions):
  return f"""\
{system_message}

{human_message}

{format_instructions}

"""

system_message_prompt = system_message_template.format()
human_message_prompt = human_message_template.format(text=text)
format_instructions_prompt = format_instructions_template.format()
prompt = format_prompt(system_message_prompt, human_message_prompt, format_instructions_prompt)
print(prompt)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.


text: This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


Format the output as JSON with the following keys:
gift
delivery_days
price_value





In [ ]:
response = get_completion(prompt)
print(response)

output_dict = json.loads(response)
output_dict.get('delivery_days')

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


2

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

chat([HumanMessage(content="Hi!")]).content

'Hello! How can I assist you today?'

In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)

gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days", description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value", description="Extract any sentences about the value or price, and output them as a comma separated Python list.")
response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
format_instructions_prompt = SystemMessage(content=format_instructions)

prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt, format_instructions_prompt])
prompt = prompt_template.format_messages(text=text)
for message in prompt: print(message.content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "`

In [ ]:
response = chat(prompt).content
print(response)

output_dict = output_parser.parse(response)
output_dict.get('delivery_days')

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


2

## Memory

In [31]:
llm = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory = memory, verbose=True)

In [32]:
print(conversation.predict(input="Hi, my name is Mert."))
print(conversation.predict(input="What is 1+1?"))
print(conversation.predict(input="What is my name?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Mert.
AI:

> Finished chain.
Hello Mert! It's nice to meet you. How can I assist you today?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Mert.
AI: Hello Mert! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.
1+1 equals 2. Is there anything else you would like to know?


> Entering new ConversationChain chain

In [13]:
print(memory.buffer, '\n')
memory.load_memory_variables({})

Human: Hi, my name is Mert.
AI: Hello Mert! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Mert. Is there anything else you would like to know or discuss? 



{'history': "Human: Hi, my name is Mert.\nAI: Hello Mert! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1 + 1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Mert. Is there anything else you would like to know or discuss?"}

In [14]:
# add aditional data to memory
memory.save_context({"input": "Hi!"}, {"output": "What's up?"})
print(memory.buffer)

Human: Hi, my name is Mert.
AI: Hello Mert! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Mert. Is there anything else you would like to know or discuss?
Human: Hi!
AI: What's up?


In [17]:
memory = ConversationBufferWindowMemory(k=1) # only keeps most recent `k` conversations

memory.save_context({"input": "Hi!"}, {"output": "What's up?"})
memory.save_context({"input": "Not much, just hanging."}, {"output": "Cool."})

print(memory.buffer)

Human: Not much, just hanging.
AI: Cool.


In [23]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30) # chops off the earlier parts of the conversation to not exceed the token limit dependent on the LLM because usually cost is determined by number of tokens

memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

print(memory.buffer)

AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


In [28]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=30) # uses the specified LLM to summarize the entire chat history which exceeds the specified number of tokens

memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

print(memory.load_memory_variables({})['history'])
# `System` is not official OpenAI system message!

System: The human expresses surprise at the AI's positive view of artificial intelligence. The AI responds with "Amazing!" and the human asks about backpropagation.
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


## Chains

In [44]:
llm = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo")

product = "Queen Size Sheet Set"
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [36]:
prompt_template = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")
chain = LLMChain(llm=llm, prompt=prompt_template) # LLM + prompt (most basic)

chain.run(product)

'Royal Comfort Bedding'

In [41]:
# ideal when expecting 1 input & returning 1 output for every chain in sequence (output of the previous chain passed as input into the next chain)
prompt_template_1 = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")
chain_1 = LLMChain(llm=llm, prompt=prompt_template_1)

prompt_template_2 = ChatPromptTemplate.from_template("Write a 20 words description for the following company:{company_name}")
chain_2 = LLMChain(llm=llm, prompt=prompt_template_2)

chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)

chain.run(product)



> Entering new SimpleSequentialChain chain...
Regal Dreams Co.
Regal Dreams Co. specializes in luxury home decor items, offering elegant and timeless pieces to elevate any living space.

> Finished chain.


'Regal Dreams Co. specializes in luxury home decor items, offering elegant and timeless pieces to elevate any living space.'

In [52]:
# sequentially executed multiple inputs & multiple outputs (more complicated)
prompt_template_1 = ChatPromptTemplate.from_template("Translate the following review to english:\n\n{Review}")
chain_1 = LLMChain(llm=llm, prompt=prompt_template_1, output_key="English_Review")

prompt_template_2 = ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence:\n\n{English_Review}")
chain_2 = LLMChain(llm=llm, prompt=prompt_template_2, output_key="summary")

prompt_template_3 = ChatPromptTemplate.from_template("What language is the following review:\n\n{Review}")
chain_3 = LLMChain(llm=llm, prompt=prompt_template_3, output_key="language")

prompt_template_4 = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
chain_4 = LLMChain(llm=llm, prompt=prompt_template_4, output_key="followup_message")

chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3, chain_4],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

chain(review)
# `run` not supported when there is not exactly one output key



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is dissatisfied with the taste and foam of the product, suspecting it may be an old batch or counterfeit.',
 'followup_message': "Je vous remercie pour votre avis sur notre produit. Nous sommes désolés que vous ayez été insatisfait de son goût et de sa mousse. Nous prenons très au sérieux la qualité de nos produits et nous aimerions en savoir plus sur votre expérience pour investiguer davantage. S'il vous plaît, contactez notre service clientèle pour que nous puissions résoudre ce problème au plus vite. Merci de nous avoir informés."}

In [58]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise \
and easy to understand manner. \
When you don't know the answer to a question you admit \
that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, \
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people, \
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence \
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist. \
You have a passion for creativity, collaboration, \
forward-thinking, confidence, strong problem-solving capabilities, \
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

# chains that will be called by router chain
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# called when the router can't decide which sub-chain to use
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

# template that will be used by LLM to route between chains
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising \
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \\ name of the prompt to use or "DEFAULT"
    "next_inputs": string \\ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not \
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(template=router_template, input_variables=["input"], output_parser=RouterOutputParser()) # parses the LLM output to determine which chain to use and what the output to that chain should be

# for routing between multiple prompt templates
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# decides on which sub-chain to route the response as an input (more complex) according to the prompt templates by passing the descriptions to an LLM
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect black body, which absorbs all incoming radiation and emits energy at all wavelengths. The distribution of this radiation follows Planck's law, which describes how the intensity of the radiation changes with temperature. Black body radiation is an important concept in physics and has applications in various fields, including astrophysics and thermodynamics."